In [29]:
import numpy as np
import mujoco as mujoco
import matplotlib.pyplot as plt
import matplotlib.animation as animate

import mujoco_funcs as mj_funcs
import mjcf_models as mj_models
import visualize_mj_funcs as mj_vis

import dyn_functions as dyn
import visualize_dyn_funcs as vis_dyn
import gen_ctrl_funcs as gen_ctrl
import analyze_ilqr_output_funcs as analyze
np.set_printoptions(precision=5, suppress=True, linewidth=100)


In [47]:
dt = 0.01
len_seq = 200
time_vec = np.arange(0,len_seq*dt, dt)
h_bar = 1.0
r_bar = 0.05
m_bar = 1.0
moi = (1/12)*h_bar*(m_bar**2 + 3*r_bar**2)
x_init = np.array([0.0,0.0,0.0, 0.0])
u_vec = np.array([0.0,0.0]) 
x_seq = np.zeros((len_seq, 4))
x_seq[0]= x_init

In [48]:
mjcf_model  = mj_models.create_MJCF_double_pend_dev(1,1,1)
mj_model, mj_render, mj_data = mj_funcs.create_mujoco_model(mjcf_model, dt)

In [49]:

nu = mj_model.nu
nx = mj_model.nv    
eps = 1e-6
A = np.zeros((2*nx, 2*nx))
B = np.zeros((2*nx,   nu))
flg_centered = False
mj_model.opt.timestep = dt
mj_data.ctrl = u_vec
mujoco.mj_forward(mj_model,mj_data)
print(mj_data.qpos)
print(mj_data.qvel)

[0. 0.]
[0. 0.]


In [56]:
dpend_sys = dyn.double_pend_rel_dyn(g=9.81, m1=m_bar, moi1=moi, d1=h_bar/2, l1=h_bar, 
                                            m2=m_bar, moi2=moi, d2=h_bar/2, l2=h_bar, 
                                        b1=0.0, b2=0.0, shoulder_act=True, elbow_act=True)
disc_dyn_func = lambda x, u: gen_ctrl.step_rk4(dpend_sys.cont_dyn_func, dt, x, u)


In [57]:
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_init, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.99874  0.00125  0.01     0.     ]
 [ 0.00168  0.9954   0.       0.01   ]
 [-0.12591  0.12541  1.       0.     ]
 [ 0.16755 -0.46001  0.       1.     ]]
A dyn: 
 [[ 0.99937  0.00063  0.01     0.     ]
 [ 0.00084  0.9977   0.       0.00999]
 [-0.12587  0.12533  0.99937  0.00063]
 [ 0.16742 -0.45978  0.00084  0.9977 ]]
B mujoco: 
 [[ 0.00017 -0.00043]
 [-0.00043  0.00136]
 [ 0.01708 -0.04265]
 [-0.04265  0.13643]]
B dyn: 
 [[ 0.00009 -0.00021]
 [-0.00021  0.00068]
 [ 0.01707 -0.04262]
 [-0.04262  0.13636]]


In [28]:
x_vec = np.array([1.0, 1.0, 0.0, 0.0])
u_vec = np.array([0.0,0.0])
mj_funcs.set_mj_ctrl_vec(mj_data, u_vec)
mj_funcs.set_mj_state_vec(mj_data, x_vec)
A_mj,  B_mj  = mj_funcs.linearize_mujoco_state_and_control(mj_model, mj_data, eps, flg_centered)    
A_dyn, B_dyn = gen_ctrl.linearize_dynamics(disc_dyn_func, x_vec, u_vec)

print('A mujoco: \n', A_mj)
print('A dyn: \n',    A_dyn)
print('B mujoco: \n', B_mj )
print('B dyn: \n',    B_dyn)

A mujoco: 
 [[ 0.999 -0.     0.01   0.   ]
 [ 0.002  1.    -0.     0.01 ]
 [-0.086 -0.017  1.     0.   ]
 [ 0.217 -0.007 -0.     1.   ]]
A dyn: 
 [[ 1.    -0.     0.01  -0.   ]
 [ 0.001  1.     0.     0.01 ]
 [-0.086 -0.017  0.999 -0.   ]
 [ 0.217 -0.007  0.003  1.   ]]
B mujoco: 
 [[ 0.    -0.   ]
 [-0.     0.001]
 [ 0.009 -0.016]
 [-0.016  0.059]]
B dyn: 
 [[ 0.    -0.   ]
 [-0.     0.   ]
 [ 0.009 -0.016]
 [-0.016  0.059]]


In [ ]:
pot_energy_seq = np.zeros((len_seq,1))
kin_energy_seq = np.zeros((len_seq,1))
tot_energy_seq = np.zeros((len_seq,1))
pot_energy_seq[0] = dpend_sys.calculate_potential_energy(x_init)
kin_energy_seq[0] = dpend_sys.calculate_kinetic_energy(x_init)
tot_energy_seq[0] = dpend_sys.calculate_total_energy(x_init)